
# 🏪 Store Sales Time Series Forecasting 📈

### 1. Introduction
#### 1.1. Problem Statement
#### 1.2. Data Description
#### 1.3. Objective
### 2. Data Exploration
#### 2.1. Importing Libraries
#### 2.2. Loading Data
#### 2.3. Data Exploration
##### 2.3.1. Univariant Analysis
##### 2.3.2. Bivariant Analysis
##### 2.3.3. Multivariant Analysis
### 3. Data Cleaning
### 4. Feature Engineering
### 5. Data Preprocessing
#### 5.1. Data Transformation
### 6. Model Building
#### 6.1. Baseline Model
### 7. Model Evaluation


We have already covered the data exploration and data cleaning parts in the previous notebook. In this notebook, we will be performing the following tasks:
* Feature Engineering
* Data Preprocessing

In [2]:
# Loading libraries
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import datetime
import os
import warnings

sns.set_context('notebook', font_scale=1.5)
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('../../input/store_sales_time_series_forecasting/train_merged.csv')
# test = pd.read_csv('../../input/store_sales_time_series_forecasting/test_merged.csv')

In [4]:
train.head()

,date,store_nbr,family,sales,onpromotion,city,state,product_type,cluster,transactions,holiday_type,locale,locale_name,description,dcoilwtico,is_holiday
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0


In [5]:
test.head()

NameError: name 'test' is not defined

## 4. Feature Engineering

* Let's create some Lag features for the target variable `Weekly_Sales`:
    * sales_1_year_ago
    * sales_1_month_ago
    * sales_2_weeks_ago
    * sales_1_week_ago

We need to convert **date** column to **datetime** format before creating lag features. 

In [6]:
train['date'] = pd.to_datetime(train.date)

In [7]:
# define a function that will return calculate the units sold number of a particular product from a particular store.
# We need to calculate this for N number of days

def get_lag_feature(data, no_of_days, return_Series= False):
    data_copy = data.copy()
    sample_1 = data_copy[['date', 'store_nbr', 'family', 'sales']]
    data_copy['NEW_DATE'] = data_copy.date + timedelta(days = no_of_days)
    data_copy['PAST_DATE'] = data_copy.date
    
    sample_2 = data_copy[['NEW_DATE','PAST_DATE', 'store_nbr', 'family', 'sales']]
    final = sample_1.merge(sample_2, how = 'left', left_on = ['date', 'store_nbr', 'family'], right_on = ['NEW_DATE', 'store_nbr', 'family'])
    final = final.drop(columns=['NEW_DATE'])
    final.fillna(0,inplace = True)
    
    if return_Series:
        return final['sales_y']
    else: return final

In [8]:
#  create a feature sales_1_year_ago
train['sales_1_year_ago'] = get_lag_feature(train, no_of_days=365, return_Series= True)

In [9]:
#  create a feature sales_1_month_ago
train['sales_1_month_ago'] = get_lag_feature(train, no_of_days=30, return_Series= True)

In [10]:
#  create a feature sales_2_week_ago
train['sales_2_week_ago'] = get_lag_feature(train, no_of_days=15, return_Series= True)

In [11]:
#  create a feature sales_1_week_ago
train['sales_1_week_ago'] = get_lag_feature(train, no_of_days=7, return_Series= True)

In [12]:
train.head()

,date,store_nbr,family,sales,onpromotion,city,state,product_type,cluster,transactions,holiday_type,locale,locale_name,description,dcoilwtico,is_holiday,sales_1_year_ago,sales_1_month_ago,sales_2_week_ago,sales_1_week_ago
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,0.0,Missing,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0


* Let's create a feature **average_in_2_months** which will be the average of daily sales for the last 2 months.

In [15]:
# make 2 columns one with 63 days difference from the week end date and another with 7 days difference
# train['2_MONTH_BEFORE'] = train.date - timedelta(days=63)
current_date = train['date'].max()
two_months_ago = current_date - pd.DateOffset(months=2)
two_months_ago

Timestamp('2017-06-15 00:00:00')

In [14]:
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()
# # calculate the average units in the period
# def get_average_sales(x):
#     data_2month = train[(train['date'] >= x['2_MONTH_BEFORE']) & (train['store_nbr'] == x['store_nbr']) & (train['family'] == x['family']) ]
#     return data_2month.sales.mean() 

train[(train['date'] >= train['2_MONTH_BEFORE'])]

,date,store_nbr,family,sales,onpromotion,city,state,product_type,cluster,transactions,...,locale,locale_name,description,dcoilwtico,is_holiday,sales_1_year_ago,sales_1_month_ago,sales_2_week_ago,sales_1_week_ago,2_MONTH_BEFORE
0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.000,0.000,0.000,0.000,2012-10-30
1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.000,0.000,0.000,0.000,2012-10-30
2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.000,0.000,0.000,0.000,2012-10-30
3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.000,0.000,0.000,0.000,2012-10-30
4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.000,0.000,0.000,0.000,2012-10-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,2155.0,...,Missing,Missing,Missing,47.57,0,394.643,571.333,470.513,358.132,2017-06-13
3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,2155.0,...,Missing,Missing,Missing,47.57,0,116.874,125.960,61.940,112.954,2017-06-13
3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,2155.0,...,Missing,Missing,Missing,47.57,0,1262.692,2041.967,1517.552,2299.715,2017-06-13
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,2155.0,...,Missing,Missing,Missing,47.57,0,10.000,0.000,56.000,170.000,2017-06-13


In [26]:
# train['average_in_2_months'] = train.progress_apply(get_average_sales,axis=1)


  0%|          | 0/3000888 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
train.head()

,date,store_nbr,family,sales,onpromotion,city,state,product_type,cluster,transactions,...,locale,locale_name,description,dcoilwtico,is_holiday,sales_1_year_ago,sales_1_month_ago,sales_2_week_ago,sales_1_week_ago,2_MONTH_BEFORE
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0,2012-10-30
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0,2012-10-30
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0,2012-10-30
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0,2012-10-30
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,0.0,...,Missing,Missing,Missing,93.14,0,0.0,0.0,0.0,0.0,2012-10-30


In [20]:
# fill the null values with 0
train.average_in_2_months.fillna(0,inplace=True)

(3000888, 21)

In [22]:
# drop the date columns that we have created, as they are of no use now.
train.drop(columns=['2_MONTH_BEFORE'], inplace=True)

,date,store_nbr,family
0,2013-01-01,1,AUTOMOTIVE
1,2013-01-01,1,BABY CARE
2,2013-01-01,1,BEAUTY
3,2013-01-01,1,BEVERAGES
4,2013-01-01,1,BOOKS
...,...,...,...
3000883,2017-08-15,9,POULTRY
3000884,2017-08-15,9,PREPARED FOODS
3000885,2017-08-15,9,PRODUCE
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES
